#Fonte: https://medium.com/turing-talks/aprendizado-por-refor%C3%A7o-4-gym-d18ac1280628
#Este exemplo foi criado por Enzo Cardeal Neves (https://medium.com/@enzocardeal) e foi adaptado por Bruno Menezes (https://github.com/brunoleomenezes) e será utilizado apenas para fins acadêmicos.

In [1]:
import numpy as np
import gym
import random
from IPython.display import clear_output
from time import sleep

In [2]:
env = gym.make("Taxi-v3").env
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



#Dimensões dos espaços

Espaço de ações e espaço de estados, respectivamente:


In [3]:
print(env.action_space)
print(env.observation_space)

Discrete(6)
Discrete(500)


#Inicializando a tabela-q

Note aqui a razão de precisarmos ter um número finito de estados possíveis. Caso contrário não seria possível mapeá-los em uma tabela


In [4]:
tabela_q = np.zeros([env.observation_space.n, env.action_space.n]) #iniciando a tabelo q com zeros

    +20 para um desembarque correto.
    -10 para um embarque ou desembarque incorreto.
    -1 para ações que não sejam as duas anteriores.

#Treinando o algoritmo

In [ ]:
#treinando o algoritmo

#aqui não existem valores "certos" ou "errados", decidimos por tentativa e erro aqueles que otimizaram o treinamento do nosso agente
alpha = 0.1
gamma = 0.6
epsilon = 0.1 #determina a chance do agente tomar uma ação aleatória, nesse caso a chance é de 10%

for i in range(1, 50001):
    estado = env.reset()

    epochs, penalidades, recompensa = 0, 0, 0 #epochs é cada episódio
    terminado = False
    
    while not terminado:
        if random.uniform(0, 1) < epsilon: #decidindo se será tomado uma ação aleatória ou se seguirá a política da tabela-q
            acao = env.action_space.sample() 
        else:
            acao = np.argmax(tabela_q[estado]) 

        proximo_estado, recompensa, terminado, info = env.step(acao) 
        
        valor_antigo = tabela_q[estado, acao]
        proximo_max = np.max(tabela_q[proximo_estado])
        
        valor_novo = (1 - alpha) * valor_antigo + alpha * (recompensa + gamma * proximo_max) #atualizando o valor de q a partir da equação de Bellman
        tabela_q[estado, acao] = valor_novo #colocando este valor na tabela-q

        if recompensa == -10: #contabilizando os embarques/desembarques errados
            penalidades += 1

        estado = proximo_estado
        epochs += 1
        
        clear_output(wait=True) #caso não queira ver o aprendizado comentar as 3 linhas seguintes, essa incluso
        env.render()
        sleep(.25)  #aumentar se quiser ver melhor o aprendizado (recomendado: .25)
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episódios: {i}")
        #sleep(1)

print("Treinamento terminado.\n")


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)


#Testando o agente

In [ ]:
#testando o algoritmo
epochs_totais, penalidades_totais = 0, 0
episodios = 100

for _ in range(episodios):
    estado = env.reset()
    epochs, penalidades, recompensa = 0, 0, 0
    
    terminado = False
    
    while not terminado:
        acao = np.argmax(tabela_q[estado])
        estado, recompensa, terminado, info = env.step(acao)

        if recompensa == -10:
            penalidades += 1

        epochs += 1
        
        clear_output(wait=True)
        env.render()
        sleep(.25)

    penalidades_totais += penalidades
    epochs_totais += epochs

print(f"Resutados depois de {episodios} episodios:")
print(f"Média de passos por episódio: {epochs_totais / episodios}")
print(f"Média de penalidades por episódio: {penalidades_totais / episodios}")